## Imports

In [11]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

## Read Data

In [2]:
data_X = pd.read_csv("data/medical/historical_X.dat", delim_whitespace=True, header=None, dtype='bool')
data_X.info()
data_X.isna().sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 130 entries, 0 to 129
dtypes: bool(130)
memory usage: 1.2 MB


0

In [3]:
data_a = pd.read_csv("data/medical/historical_A.dat", delim_whitespace=True, header=None, dtype='bool')
data_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       10000 non-null  bool 
dtypes: bool(1)
memory usage: 9.9 KB


In [4]:
data_y = pd.read_csv("data/medical/historical_Y.dat", delim_whitespace=True, header=None, dtype='bool')
data_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       10000 non-null  bool 
dtypes: bool(1)
memory usage: 9.9 KB


## Modeling

In [5]:
X = tf.convert_to_tensor(data_X.values, dtype=tf.float32)
a = tf.convert_to_tensor(data_a.values, dtype=tf.float32)
y = tf.convert_to_tensor(data_y.values, dtype=tf.float32)

In [6]:
def gen_logistic_model(X, a, prior_mean=0, prior_scale=1):
    num_of_features = data_X.shape[1]
    mean = tf.cast(prior_mean, tf.float32)
    scale = tf.cast(prior_scale, tf.float32)

    return tfd.JointDistributionNamed({
        'alpha_1': tfd.Sample(tfd.Normal(loc=mean, scale=scale), sample_shape=[1, 1]),
        'alpha_2': tfd.Sample(tfd.Normal(loc=mean, scale=scale), sample_shape=[1, 1]),
        'beta_1': tfd.Independent(
            tfd.Normal(
                loc=tf.zeros([num_of_features, 1]) + mean,
                scale=tf.ones([num_of_features, 1]) * scale),
            reinterpreted_batch_ndims=2),
        'beta_2': tfd.Independent(
            tfd.Normal(
                loc=tf.zeros([num_of_features, 1]) + mean,
                scale=tf.ones([num_of_features, 1]) * scale),
            reinterpreted_batch_ndims=2),
        'y': lambda alpha_1, alpha_2, beta_1, beta_2: tfd.Independent(
            tfd.Bernoulli(
                logits=(tf.matmul(X, beta_1) + alpha_1)*a +
                    (tf.matmul(X, beta_2) + alpha_2)*(1.-a)
            ),
            reinterpreted_batch_ndims=2)
    })

In [7]:
log_model = gen_logistic_model(X, a)

print(log_model.resolve_graph())

for key in log_model.sample_distributions()[    0]:
    print(f"{key}:\n\t{log_model.sample_distributions()[0][key]}")

prior_predictive_samples = log_model.sample()
log_model.log_prob_parts(prior_predictive_samples)

(('beta_2', ()), ('beta_1', ()), ('alpha_2', ()), ('alpha_1', ()), ('y', ('alpha_1', 'alpha_2', 'beta_1', 'beta_2')))
beta_2:
	tfp.distributions.Independent("IndependentNormal", batch_shape=[], event_shape=[130, 1], dtype=float32)
beta_1:
	tfp.distributions.Independent("IndependentNormal", batch_shape=[], event_shape=[130, 1], dtype=float32)
alpha_2:
	tfp.distributions.Sample("SampleNormal", batch_shape=[], event_shape=[1, 1], dtype=float32)
alpha_1:
	tfp.distributions.Sample("SampleNormal", batch_shape=[], event_shape=[1, 1], dtype=float32)
y:
	tfp.distributions.Independent("JointDistributionNamed_sample_distributions_IndependentJointDistributionNamed_sample_distributions_Bernoulli", batch_shape=[], event_shape=[10000, 1], dtype=int32)


{'beta_2': <tf.Tensor: shape=(), dtype=float32, numpy=-189.02846>,
 'beta_1': <tf.Tensor: shape=(), dtype=float32, numpy=-179.86261>,
 'alpha_2': <tf.Tensor: shape=(), dtype=float32, numpy=-3.2453175>,
 'alpha_1': <tf.Tensor: shape=(), dtype=float32, numpy=-1.0220878>,
 'y': <tf.Tensor: shape=(), dtype=float32, numpy=-853.6043>}

In [8]:
_ = log_model.sample()
_ = log_model.sample(4)
sample = log_model.sample([3, 4])
log_model.log_prob(sample)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-1917.4045, -2425.7593, -2302.3567, -1057.8467],
       [-1138.6799, -1335.8071, -1450.4109, -2549.8313],
       [-1976.6567, -1588.7765, -2541.5554, -2535.4165]], dtype=float32)>

In [9]:
@tf.function(autograph=False, experimental_compile=True)
def run_chain(init_state, step_size, target_log_prob_fn, unconstraining_bijectors,
              num_steps=500, burnin=50):

  def trace_fn(_, pkr):
    return (
        pkr.inner_results.inner_results.target_log_prob,
        pkr.inner_results.inner_results.leapfrogs_taken,
        pkr.inner_results.inner_results.has_divergence,
        pkr.inner_results.inner_results.energy,
        pkr.inner_results.inner_results.log_accept_ratio
           )

  kernel = tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=tfp.mcmc.NoUTurnSampler(
      target_log_prob_fn,
      step_size=step_size),
    bijector=unconstraining_bijectors)

  hmc = tfp.mcmc.DualAveragingStepSizeAdaptation(
    inner_kernel=kernel,
    num_adaptation_steps=burnin,
    step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(
        inner_results=pkr.inner_results._replace(step_size=new_step_size)),
    step_size_getter_fn=lambda pkr: pkr.inner_results.step_size,
    log_accept_prob_getter_fn=lambda pkr: pkr.inner_results.log_accept_ratio
  )

  # Sampling from the chain.
  chain_state, sampler_stat = tfp.mcmc.sample_chain(
      num_results=num_steps,
      num_burnin_steps=burnin,
      current_state=init_state,
      kernel=hmc,
      trace_fn=trace_fn)
  return chain_state, sampler_stat

In [20]:
def logp_batch(*args):
    return log_model.log_prob(args)

sample = log_model.sample(4)
sample['y'] = y
print(log_model.log_prob(sample).numpy()) 
print(logp_batch(*list(sample.values())).numpy())

[-26747.398 -18943.99  -33720.99  -31130.422]


TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [15]:
nchain = 4
sample = log_model.sample(nchain)
init_state = list(sample.values())[:-1]
step_size = tf.repeat(.1, len(init_state))
target_log_prob_fn = lambda *x: mdl_ols_batch.log_prob(x + (y, ))

# bijector to map contrained parameters to real
unconstraining_bijectors = [
    tfb.Identity(),
    tfb.Identity(),
]

samples, sampler_stat = run_chain(
    init_state, step_size, target_log_prob_fn, unconstraining_bijectors)

tf.Tensor([0.1 0.1 0.1 0.1], shape=(4,), dtype=float32)
Instructions for updating:
The `seed` argument is deprecated (but will work until removed). Pass seed to `tfp.mcmc.sample_chain` instead.


NameError: name 'mdl_ols_batch' is not defined